# 12: Tier data part 2 - get which Tier trips started/ended at stations

In this notebook, we will look, which Tier trips started within one of our stop buffer zones. If a trip started (or ended or both) in such a buffer zone, the information will be added to the Tier dataset
- start station 
    - id
    - name
    - buffer zone
- end station
    - id
    - name
    - buffer zone

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
from shapely import Point, wkb, wkt

In [2]:
data_month_single = '7'
data_month_double = '07'

## Load and prepare data

In [3]:
# load tier trip data
tier_trips = pd.DataFrame(pd.read_parquet("/../11_tier_part1.parquet", engine="pyarrow"))
tier_trips

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,start_location,end_location,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters
801621,497085150,2023-01-02 16:10:00,2023-01-02 16:20:00,b'\x01\x01\x00\x00\x00_^\x80}t\\I@\xc7\xf4\x84...,b'\x01\x01\x00\x00\x00y\x92t\xcd\xe4[I@\x8d\x0...,e-scooter,0 days 00:10:00,0.617222,617.221763
801629,497085158,2023-01-02 21:25:00,2023-01-02 21:40:00,"b""\x01\x01\x00\x00\x00x*\xe0\x9e\xe7[I@'2s\x81...","b""\x01\x01\x00\x00\x00\x8d]\xa2zk\\I@*9'\xf6\x...",e-scooter,0 days 00:15:00,0.699559,699.558708
801631,497085160,2023-01-02 21:50:00,2023-01-02 22:00:00,b'\x01\x01\x00\x00\x00\xfe\x9c\x82\xfcl\\I@\xf...,b'\x01\x01\x00\x00\x00\xb2\x11\x88\xd7\xf5[I@\...,e-scooter,0 days 00:10:00,0.766417,766.417469
801637,497085166,2023-01-03 06:50:00,2023-01-03 07:40:00,b'\x01\x01\x00\x00\x00G\x8f\xdf\xdb\xf4[I@\xa8...,b'\x01\x01\x00\x00\x00:z\xfc\xde\xa6aI@M\xd7\x...,e-scooter,0 days 00:50:00,10.615185,10615.184930
801643,497085172,2023-01-03 14:50:00,2023-01-03 15:15:00,b'\x01\x01\x00\x00\x00R\n\xba\xbd\xa4aI@\t\xfb...,b'\x01\x01\x00\x00\x00\x1b\xf6{b\x9d^I@\x8b\x8...,e-scooter,0 days 00:25:00,5.571567,5571.567347
...,...,...,...,...,...,...,...,...,...
4807552,1006547193,2023-12-29 10:30:00,2023-12-29 11:00:00,b'\x01\x01\x00\x00\x00;\x01M\x84\r[I@\x1d\xaa)...,b'\x01\x01\x00\x00\x00\xec\x12\xd5[\x03_I@\x87...,e-scooter,0 days 00:30:00,8.561807,8561.806757
4807558,1006547199,2023-12-29 11:35:00,2023-12-29 12:05:00,b'\x01\x01\x00\x00\x00\x1a\xde\xac\xc1\xfb^I@[...,b'\x01\x01\x00\x00\x00;\xe3\xfb\xe2R[I@@3\x88\...,e-scooter,0 days 00:30:00,8.172812,8172.812192
4807582,1006547223,2023-12-28 15:15:00,2023-12-28 15:20:00,b'\x01\x01\x00\x00\x00\xa2\x9chW!]I@\x8b\x89\x...,b'\x01\x01\x00\x00\x00\xf3\xab9@0]I@g|_\\\xaa\...,e-scooter,0 days 00:05:00,0.115253,115.252520
4807597,1006547238,2023-12-28 16:00:00,2023-12-28 16:10:00,b'\x01\x01\x00\x00\x00\xc0>:u\xe5_I@ \xb5\x89\...,b'\x01\x01\x00\x00\x00|\x99(B\xea^I@`\xe7\xa6\...,e-scooter,0 days 00:10:00,0.849810,849.809943


In [4]:
# transform geometry
point_start = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier_trips["start_location"], crs=4326))
point_end = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier_trips["end_location"], crs=4326))

In [5]:
# drop geometry columns in wrong format
tier_trips = tier_trips.drop(columns=["start_location", "end_location"])

In [6]:
# replace geometry columns with correct format
tier_trips['start_location'] = point_start
tier_trips['end_location'] = point_end

In [7]:
# change x and y coordinates
tier_trips['start_location'] = tier_trips['start_location'].apply(lambda point: Point(point.y, point.x))
tier_trips['end_location'] = tier_trips['end_location'].apply(lambda point: Point(point.y, point.x))

In [8]:
tier_trips.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354421 entries, 801621 to 4807604
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype          
---  ------                    --------------   -----          
 0   tier_trips_id             354421 non-null  int64          
 1   tier_trips_start_time     354421 non-null  datetime64[ns] 
 2   tier_trips_end_time       354421 non-null  datetime64[ns] 
 3   vehicle type              354421 non-null  object         
 4   trip_duration             354421 non-null  timedelta64[ns]
 5   trip_distance_kilometers  354421 non-null  float64        
 6   trip_distance_meters      354421 non-null  float64        
 7   start_location            354421 non-null  object         
 8   end_location              354421 non-null  object         
dtypes: datetime64[ns](2), float64(2), int64(1), object(3), timedelta64[ns](1)
memory usage: 27.0+ MB


In [9]:
# tranform start location to geometry
tier_trips = gpd.GeoDataFrame(tier_trips, geometry='start_location')

In [10]:
# transform end_location to geometry
tier_trips = gpd.GeoDataFrame(tier_trips, geometry='end_location')

In [11]:
# set GeoDataFrame geometry back to start_location
tier_trips = gpd.GeoDataFrame(tier_trips, geometry='start_location')

In [12]:
tier_trips.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 354421 entries, 801621 to 4807604
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype          
---  ------                    --------------   -----          
 0   tier_trips_id             354421 non-null  int64          
 1   tier_trips_start_time     354421 non-null  datetime64[ns] 
 2   tier_trips_end_time       354421 non-null  datetime64[ns] 
 3   vehicle type              354421 non-null  object         
 4   trip_duration             354421 non-null  timedelta64[ns]
 5   trip_distance_kilometers  354421 non-null  float64        
 6   trip_distance_meters      354421 non-null  float64        
 7   start_location            354421 non-null  geometry       
 8   end_location              354421 non-null  geometry       
dtypes: datetime64[ns](2), float64(2), geometry(2), int64(1), object(1), timedelta64[ns](1)
memory usage: 27.0+ MB


In [14]:
#vrs_data = pd.DataFrame(pd.read_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/10_VRS_data_clean.parquet", engine="pyarrow"))
#vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/19_Merged_with_Weather_Holiday.csv"))
vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/Processed Data/07/11_VRs_clean_NEU.csv"))

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_1872/3719310214.py:3: DtypeWarning: Columns (5,19) have mixed types. Specify dtype option on import or set low_memory=False.
  vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/Processed Data/07/11_VRs_clean_NEU.csv"))


In [15]:
# transform geometry
vrs_data['station_point'] = vrs_data['station_point'].apply(wkt.loads)

In [16]:
vrs_data['buffer_zone'] = shapely.wkt.loads(vrs_data['buffer_zone'])

In [25]:
vrs_data = gpd.GeoDataFrame(vrs_data, geometry='station_point')

In [26]:
vrs_data = gpd.GeoDataFrame(vrs_data, geometry='buffer_zone')

In [27]:
vrs_data = gpd.GeoDataFrame(vrs_data, geometry='station_point')

In [18]:
# transform geometry (parquet)
#point = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(vrs_data["station_point"], crs=4326))
#buffer = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(vrs_data["buffer_zone"], crs=4326))

TypeError: Expected bytes or string, got Point

In [ ]:
# drop geometry columns in wrong format (parquet)
##vrs_data= vrs_data.drop(columns=["station_point", "buffer_zone"])

In [ ]:
# replace geometry columns with correct format (parquet)
#vrs_data['station_point'] = point
#vrs_data['buffer_zone'] = buffer

In [28]:
vrs_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2564861 entries, 0 to 2564860
Data columns (total 31 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   Unnamed: 0.2              int64   
 1   Unnamed: 0.1              int64   
 2   Unnamed: 0                int64   
 3   route_id                  int64   
 4   agency_id                 int64   
 5   route_short_name          object  
 6   route_type                int64   
 7   route_type_name           object  
 8   agency_name               object  
 9   service_id                int64   
 10  trip_id                   object  
 11  trip_headsign             object  
 12  direction_id              int64   
 13  shape_id                  int64   
 14  stop_id                   int64   
 15  actual_arrival_time       object  
 16  actual_departure_time     object  
 17  vrs_timestamp             int64   
 18  stop_sequence             int64   
 19  stop_headsign             object  

In [29]:
vrs_data

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,...,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,date,weekday,transfer_stop
0,93805,93805,93805,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,...,POINT (7.04787 50.73740),"POLYGON ((7.05062 50.73773, 7.05066 50.73756, ...",2023-07-03 03:48:00,2023-07-03 03:48:00,NaN,0 days,1,2023-07-03,Monday,True
1,65490372,65490372,65490372,537,8,537,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,64,...,POINT (7.09713 50.73201),"POLYGON ((7.09988 50.73234, 7.09992 50.73217, ...",2023-07-03 05:23:00,2023-07-03 05:23:00,0 days,NaN,1,2023-07-03,Monday,False
2,65491200,65491200,65491200,537,8,537,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,64,...,POINT (7.09713 50.73201),"POLYGON ((7.09988 50.73234, 7.09992 50.73217, ...",2023-07-04 05:23:00,2023-07-04 05:23:00,0 days,NaN,1,2023-07-04,Tuesday,False
3,65491982,65491982,65491982,537,8,537,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,64,...,POINT (7.09713 50.73201),"POLYGON ((7.09988 50.73234, 7.09992 50.73217, ...",2023-07-05 05:23:00,2023-07-05 05:23:00,0 days,NaN,1,2023-07-05,Wednesday,False
4,65492787,65492787,65492787,537,8,537,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,64,...,POINT (7.09713 50.73201),"POLYGON ((7.09988 50.73234, 7.09992 50.73217, ...",2023-07-06 05:23:00,2023-07-06 05:23:00,0 days,NaN,1,2023-07-06,Thursday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564856,115985,115985,115985,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,...,POINT (7.09765 50.73628),"POLYGON ((7.10040 50.73662, 7.10044 50.73644, ...",2023-07-02 19:39:00,2023-07-02 19:39:00,NaN,NaN,1,2023-07-02,Sunday,False
2564857,115984,115984,115984,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,...,POINT (7.09487 50.73377),"POLYGON ((7.09762 50.73411, 7.09766 50.73393, ...",2023-07-02 19:37:00,2023-07-02 19:37:00,NaN,NaN,1,2023-07-02,Sunday,True
2564858,115986,115986,115986,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,...,POINT (7.10248 50.73747),"POLYGON ((7.10523 50.73780, 7.10527 50.73763, ...",2023-07-02 19:41:00,2023-07-02 19:41:00,NaN,NaN,1,2023-07-02,Sunday,True
2564859,115987,115987,115987,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,...,POINT (7.11744 50.73923),"POLYGON ((7.12019 50.73957, 7.12023 50.73939, ...",2023-07-02 19:44:00,2023-07-02 19:44:00,NaN,NaN,1,2023-07-02,Sunday,True


## Check if Tier trip start and/or ends within a buffer zone

In [30]:
# get all unique stops with their stop id, stop_name and buffer_zone that are in the vrs_data
unique_stops = vrs_data[['stop_id', 'stop_name', 'buffer_zone']].drop_duplicates()

In [31]:
unique_stops

,stop_id,stop_name,buffer_zone
0,697,Bonn Dransdorf,"POLYGON ((7.05062 50.73773, 7.05066 50.73756, ..."
1,687,Bonn Hbf,"POLYGON ((7.09988 50.73234, 7.09992 50.73217, ..."
22,1103,Bonn Markt,"POLYGON ((7.10538 50.73501, 7.10542 50.73483, ..."
43,1112,Bonn Stiftsplatz,"POLYGON ((7.10435 50.73943, 7.10439 50.73926, ..."
64,1115,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,"POLYGON ((7.10523 50.73780, 7.10527 50.73763, ..."
...,...,...,...
2497659,5569,Bonn Potsdamer Platz,"POLYGON ((7.08300 50.74314, 7.08304 50.74296, ..."
2520484,8087,Bonn Grüner Weg,"POLYGON ((7.17906 50.70518, 7.17910 50.70501, ..."
2524262,1984,Bonn Chemische Institute,"POLYGON ((7.07545 50.73465, 7.07549 50.73447, ..."
2524345,5567,Bonn Mva Bonn,"POLYGON ((7.08016 50.73332, 7.08019 50.73315, ..."


In [32]:
import folium

# Create a map centered around Bonn
map_buffer_zones = folium.Map(location=[50.7374, 7.0982], zoom_start=13)

# Iterate over each row in unique_stops
for index, row in unique_stops.iterrows():
    # Get the buffer zone geometry
    buffer_zone = row['buffer_zone']
    
    # Convert the buffer zone to GeoJSON format
    buffer_zone_geojson = buffer_zone.__geo_interface__
    
    # Create a GeoJSON layer for the buffer zone and add it to the map
    folium.GeoJson(buffer_zone_geojson).add_to(map_buffer_zones)

# Display the map
map_buffer_zones

In [33]:
# check if trip starts in a buffer zone

# Create empty columns in tier_data
tier_trips['start_stop_id'] = np.nan
tier_trips['start_stop_name'] = np.nan
tier_trips['start_buffer_zone'] = np.nan

# Iterate over each row in tier_data
for index, row in tier_trips.iterrows():
    # Get the coordinates of tier trips start
    tier_trips_start_coords = row['start_location']	
    
    # Check if tier trips start coordinates lie within a buffer_zone in unique_stops
    for stop_index, stop_row in unique_stops.iterrows():
        buffer_zone = stop_row['buffer_zone']
        if buffer_zone.contains(tier_trips_start_coords):
            # If true, add the stop_id, stop_name, buffer_zone to the row in tier_data
            tier_trips.at[index, 'start_stop_id'] = stop_row['stop_id']
            tier_trips.at[index, 'start_stop_name'] = stop_row['stop_name']
            tier_trips.at[index, 'start_buffer_zone'] = buffer_zone
            break

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_1872/796474089.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Bonn Ramersdorf' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tier_trips.at[index, 'start_stop_name'] = stop_row['stop_name']
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_1872/796474089.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'POLYGON ((7.157885983913106 50.72271013887745, 7.157924171995828 50.72253736779311, 7.157935498361684 50.72236305258962, 7.157919854233708 50.72218887204915, 7.157877390581811 50.722016503637626, 7.157808516657013 50.72184760734921, 7.157713896038496 50.72168380972014, 7.15759444023199 50.72152668816523, 7.1574512998816 50.72137775578825, 7.157285853680024 50.72123844681242, 7.157099695084369 50.721110102

In [34]:
# check if trip ends in a buffer zone

# Create empty columns in tier_data
tier_trips['end_stop_id'] = np.nan
tier_trips['end_stop_name'] = np.nan
tier_trips['end_buffer_zone'] = np.nan

# Iterate over each row in tier_data
for index, row in tier_trips.iterrows():
    # Get the coordinates of tier_trips_end
    tier_trips_end_coords = row['end_location']	
    
    # Check if tier_trips_end coordinates lie within a buffer_zone in unique_stops
    for stop_index, stop_row in unique_stops.iterrows():
        buffer_zone = stop_row['buffer_zone']
        if buffer_zone.contains(tier_trips_end_coords):
            # If true, add the stop_id, stop_name, buffer_zone to the row in tier_data
            tier_trips.at[index, 'end_stop_id'] = stop_row['stop_id']
            tier_trips.at[index, 'end_stop_name'] = stop_row['stop_name']
            tier_trips.at[index, 'end_buffer_zone'] = buffer_zone
            break

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_1872/2949096388.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Bonn Ramersdorf' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tier_trips.at[index, 'end_stop_name'] = stop_row['stop_name']
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_1872/2949096388.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'POLYGON ((7.157885983913106 50.72271013887745, 7.157924171995828 50.72253736779311, 7.157935498361684 50.72236305258962, 7.157919854233708 50.72218887204915, 7.157877390581811 50.722016503637626, 7.157808516657013 50.72184760734921, 7.157713896038496 50.72168380972014, 7.15759444023199 50.72152668816523, 7.1574512998816 50.72137775578825, 7.157285853680024 50.72123844681242, 7.157099695084369 50.721110102

In [35]:
tier_trips

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone
801621,497085150,2023-01-02 16:10:00,2023-01-02 16:20:00,e-scooter,0 days 00:10:00,0.617222,617.221763,POINT (7.15637 50.72230),POINT (7.15981 50.71792),1584.0,Bonn Ramersdorf,"POLYGON ((7.157885983913106 50.72271013887745,...",NaN,NaN,NaN
801629,497085158,2023-01-02 21:25:00,2023-01-02 21:40:00,e-scooter,0 days 00:15:00,0.699559,699.558708,POINT (7.15996 50.71801),POINT (7.15509 50.72203),NaN,NaN,NaN,1584.0,Bonn Ramersdorf,"POLYGON ((7.157885983913106 50.72271013887745,..."
801631,497085160,2023-01-02 21:50:00,2023-01-02 22:00:00,e-scooter,0 days 00:10:00,0.766417,766.417469,POINT (7.15508 50.72208),POINT (7.16095 50.71844),1584.0,Bonn Ramersdorf,"POLYGON ((7.157885983913106 50.72271013887745,...",NaN,NaN,NaN
801637,497085166,2023-01-03 06:50:00,2023-01-03 07:40:00,e-scooter,0 days 00:50:00,10.615185,10615.184930,POINT (7.16090 50.71841),POINT (7.07626 50.76291),NaN,NaN,NaN,1400.0,Bonn Margarethenplatz,"POLYGON ((7.078621495736716 50.76457120653019,..."
801643,497085172,2023-01-03 14:50:00,2023-01-03 15:15:00,e-scooter,0 days 00:25:00,5.571567,5571.567347,POINT (7.07624 50.76284),POINT (7.12050 50.73918),1400.0,Bonn Margarethenplatz,"POLYGON ((7.078621495736716 50.76457120653019,...",1501.0,Bonn Beuel Rathaus,"POLYGON ((7.123213568333682 50.73970703696707,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4807552,1006547193,2023-12-29 10:30:00,2023-12-29 11:00:00,e-scooter,0 days 00:30:00,8.561807,8561.806757,POINT (7.16819 50.71135),POINT (7.09758 50.74229),NaN,NaN,NaN,1162.0,Bonn Rosental,"POLYGON ((7.098849425567842 50.74221362370187,..."
4807558,1006547199,2023-12-29 11:35:00,2023-12-29 12:05:00,e-scooter,0 days 00:30:00,8.172812,8172.812192,POINT (7.09716 50.74206),POINT (7.16496 50.71347),1162.0,Bonn Rosental,"POLYGON ((7.098849425567842 50.74221362370187,...",NaN,NaN,NaN
4807582,1006547223,2023-12-28 15:15:00,2023-12-28 15:20:00,e-scooter,0 days 00:05:00,0.115253,115.252520,POINT (7.14229 50.72758),POINT (7.14323 50.72803),1585.0,Bonn Küdinghoven,"POLYGON ((7.144659133291916 50.7279374755449, ...",1585.0,Bonn Küdinghoven,"POLYGON ((7.144659133291916 50.7279374755449, ..."
4807597,1006547238,2023-12-28 16:00:00,2023-12-28 16:10:00,e-scooter,0 days 00:10:00,0.849810,849.809943,POINT (7.15428 50.74919),POINT (7.15354 50.74152),2304.0,Bonn Alte Schulstr.,"POLYGON ((7.155929606106404 50.75078631690526,...",2307.0,Bonn Holtorfer Str.,"POLYGON ((7.153668054043429 50.74182832821801,..."


In [36]:
# look at tier trips that start and end in buffer zones
test_start_stop = tier_trips[(tier_trips['start_stop_name'].notna()) & (tier_trips['end_stop_name'].notna()) & (tier_trips['start_stop_name'] != tier_trips['end_stop_name'])]
test_start_stop

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone
801643,497085172,2023-01-03 14:50:00,2023-01-03 15:15:00,e-scooter,0 days 00:25:00,5.571567,5571.567347,POINT (7.07624 50.76284),POINT (7.12050 50.73918),1400.0,Bonn Margarethenplatz,"POLYGON ((7.078621495736716 50.76457120653019,...",1501.0,Bonn Beuel Rathaus,"POLYGON ((7.123213568333682 50.73970703696707,..."
801655,497085184,2023-01-03 22:00:00,2023-01-03 22:25:00,e-scooter,0 days 00:25:00,3.785951,3785.951216,POINT (7.12027 50.73780),POINT (7.11169 50.70460),1501.0,Bonn Beuel Rathaus,"POLYGON ((7.123213568333682 50.73970703696707,...",1286.0,Bonn Quirinusplatz,"POLYGON ((7.11445943876344 50.705109080519, 7...."
801662,497085191,2023-01-05 19:15:00,2023-01-05 19:40:00,e-scooter,0 days 00:25:00,3.431713,3431.712656,POINT (7.11370 50.70820),POINT (7.11770 50.73904),1271.0,Bonn Wasserland,"POLYGON ((7.119102702665999 50.70867798782034,...",1500.0,Bonn Konrad-Adenauer-Platz,"POLYGON ((7.120193524138334 50.73956510756546,..."
801667,497085196,2023-01-05 20:20:00,2023-01-05 20:35:00,e-scooter,0 days 00:15:00,2.056873,2056.873148,POINT (7.11783 50.73927),POINT (7.13607 50.74233),1500.0,Bonn Konrad-Adenauer-Platz,"POLYGON ((7.120193524138334 50.73956510756546,...",1128.0,Bonn Gartenstr.,"POLYGON ((7.136276932614801 50.74327574547151,..."
801674,497085203,2023-01-05 21:55:00,2023-01-05 22:10:00,e-scooter,0 days 00:15:00,2.092784,2092.783720,POINT (7.13623 50.74229),POINT (7.11999 50.73270),1543.0,Bonn Röhfeldstr.,POLYGON ((7.138459925632575 50.742714691313466...,1116.0,Bonn Johannesstr.,POLYGON ((7.121399231769847 50.734331054470715...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4807525,1006547166,2023-12-30 08:30:00,2023-12-30 08:40:00,e-scooter,0 days 00:10:00,1.498038,1498.038059,POINT (7.08621 50.74504),POINT (7.09776 50.73805),1181.0,Bonn Am Nordpark,"POLYGON ((7.088504512030215 50.74578988467313,...",1102.0,Bonn Friedensplatz,"POLYGON ((7.1004031160749 50.736616560650745, ..."
4807532,1006547173,2023-12-30 16:30:00,2023-12-30 16:35:00,e-scooter,0 days 00:05:00,1.088905,1088.904507,POINT (7.09806 50.73763),POINT (7.08827 50.73740),1102.0,Bonn Friedensplatz,"POLYGON ((7.1004031160749 50.736616560650745, ...",1153.0,Bonn Heerstr.,"POLYGON ((7.092989048773444 50.73697373726556,..."
4807537,1006547178,2023-12-30 17:20:00,2023-12-30 17:30:00,e-scooter,0 days 00:10:00,1.526509,1526.509477,POINT (7.08824 50.73730),POINT (7.10197 50.73723),1153.0,Bonn Heerstr.,"POLYGON ((7.092989048773444 50.73697373726556,...",1115.0,Bonn Bertha-Von-Suttner-Pl./Beethovenhaus,POLYGON ((7.1052342430250555 50.73780245225033...
4807597,1006547238,2023-12-28 16:00:00,2023-12-28 16:10:00,e-scooter,0 days 00:10:00,0.849810,849.809943,POINT (7.15428 50.74919),POINT (7.15354 50.74152),2304.0,Bonn Alte Schulstr.,"POLYGON ((7.155929606106404 50.75078631690526,...",2307.0,Bonn Holtorfer Str.,"POLYGON ((7.153668054043429 50.74182832821801,..."


In [37]:
tier_trips.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 354421 entries, 801621 to 4807604
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype          
---  ------                    --------------   -----          
 0   tier_trips_id             354421 non-null  int64          
 1   tier_trips_start_time     354421 non-null  datetime64[ns] 
 2   tier_trips_end_time       354421 non-null  datetime64[ns] 
 3   vehicle type              354421 non-null  object         
 4   trip_duration             354421 non-null  timedelta64[ns]
 5   trip_distance_kilometers  354421 non-null  float64        
 6   trip_distance_meters      354421 non-null  float64        
 7   start_location            354421 non-null  geometry       
 8   end_location              354421 non-null  geometry       
 9   start_stop_id             281225 non-null  float64        
 10  start_stop_name           281225 non-null  object         
 11  start_buffer_zone         281225 non-null  

In [38]:
tier_trips

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone
801621,497085150,2023-01-02 16:10:00,2023-01-02 16:20:00,e-scooter,0 days 00:10:00,0.617222,617.221763,POINT (7.15637 50.72230),POINT (7.15981 50.71792),1584.0,Bonn Ramersdorf,"POLYGON ((7.157885983913106 50.72271013887745,...",NaN,NaN,NaN
801629,497085158,2023-01-02 21:25:00,2023-01-02 21:40:00,e-scooter,0 days 00:15:00,0.699559,699.558708,POINT (7.15996 50.71801),POINT (7.15509 50.72203),NaN,NaN,NaN,1584.0,Bonn Ramersdorf,"POLYGON ((7.157885983913106 50.72271013887745,..."
801631,497085160,2023-01-02 21:50:00,2023-01-02 22:00:00,e-scooter,0 days 00:10:00,0.766417,766.417469,POINT (7.15508 50.72208),POINT (7.16095 50.71844),1584.0,Bonn Ramersdorf,"POLYGON ((7.157885983913106 50.72271013887745,...",NaN,NaN,NaN
801637,497085166,2023-01-03 06:50:00,2023-01-03 07:40:00,e-scooter,0 days 00:50:00,10.615185,10615.184930,POINT (7.16090 50.71841),POINT (7.07626 50.76291),NaN,NaN,NaN,1400.0,Bonn Margarethenplatz,"POLYGON ((7.078621495736716 50.76457120653019,..."
801643,497085172,2023-01-03 14:50:00,2023-01-03 15:15:00,e-scooter,0 days 00:25:00,5.571567,5571.567347,POINT (7.07624 50.76284),POINT (7.12050 50.73918),1400.0,Bonn Margarethenplatz,"POLYGON ((7.078621495736716 50.76457120653019,...",1501.0,Bonn Beuel Rathaus,"POLYGON ((7.123213568333682 50.73970703696707,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4807552,1006547193,2023-12-29 10:30:00,2023-12-29 11:00:00,e-scooter,0 days 00:30:00,8.561807,8561.806757,POINT (7.16819 50.71135),POINT (7.09758 50.74229),NaN,NaN,NaN,1162.0,Bonn Rosental,"POLYGON ((7.098849425567842 50.74221362370187,..."
4807558,1006547199,2023-12-29 11:35:00,2023-12-29 12:05:00,e-scooter,0 days 00:30:00,8.172812,8172.812192,POINT (7.09716 50.74206),POINT (7.16496 50.71347),1162.0,Bonn Rosental,"POLYGON ((7.098849425567842 50.74221362370187,...",NaN,NaN,NaN
4807582,1006547223,2023-12-28 15:15:00,2023-12-28 15:20:00,e-scooter,0 days 00:05:00,0.115253,115.252520,POINT (7.14229 50.72758),POINT (7.14323 50.72803),1585.0,Bonn Küdinghoven,"POLYGON ((7.144659133291916 50.7279374755449, ...",1585.0,Bonn Küdinghoven,"POLYGON ((7.144659133291916 50.7279374755449, ..."
4807597,1006547238,2023-12-28 16:00:00,2023-12-28 16:10:00,e-scooter,0 days 00:10:00,0.849810,849.809943,POINT (7.15428 50.74919),POINT (7.15354 50.74152),2304.0,Bonn Alte Schulstr.,"POLYGON ((7.155929606106404 50.75078631690526,...",2307.0,Bonn Holtorfer Str.,"POLYGON ((7.153668054043429 50.74182832821801,..."


In [39]:
# transform start_buffer_zone to string
tier_trips['start_buffer_zone'] = str(tier_trips['start_buffer_zone'])
tier_trips['end_buffer_zone'] = str(tier_trips['end_buffer_zone'])

In [40]:
tier_trips

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone
801621,497085150,2023-01-02 16:10:00,2023-01-02 16:20:00,e-scooter,0 days 00:10:00,0.617222,617.221763,POINT (7.15637 50.72230),POINT (7.15981 50.71792),1584.0,Bonn Ramersdorf,801621 POLYGON ((7.157885983913106 50.7227...,NaN,NaN,801621 ...
801629,497085158,2023-01-02 21:25:00,2023-01-02 21:40:00,e-scooter,0 days 00:15:00,0.699559,699.558708,POINT (7.15996 50.71801),POINT (7.15509 50.72203),NaN,NaN,801621 POLYGON ((7.157885983913106 50.7227...,1584.0,Bonn Ramersdorf,801621 ...
801631,497085160,2023-01-02 21:50:00,2023-01-02 22:00:00,e-scooter,0 days 00:10:00,0.766417,766.417469,POINT (7.15508 50.72208),POINT (7.16095 50.71844),1584.0,Bonn Ramersdorf,801621 POLYGON ((7.157885983913106 50.7227...,NaN,NaN,801621 ...
801637,497085166,2023-01-03 06:50:00,2023-01-03 07:40:00,e-scooter,0 days 00:50:00,10.615185,10615.184930,POINT (7.16090 50.71841),POINT (7.07626 50.76291),NaN,NaN,801621 POLYGON ((7.157885983913106 50.7227...,1400.0,Bonn Margarethenplatz,801621 ...
801643,497085172,2023-01-03 14:50:00,2023-01-03 15:15:00,e-scooter,0 days 00:25:00,5.571567,5571.567347,POINT (7.07624 50.76284),POINT (7.12050 50.73918),1400.0,Bonn Margarethenplatz,801621 POLYGON ((7.157885983913106 50.7227...,1501.0,Bonn Beuel Rathaus,801621 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4807552,1006547193,2023-12-29 10:30:00,2023-12-29 11:00:00,e-scooter,0 days 00:30:00,8.561807,8561.806757,POINT (7.16819 50.71135),POINT (7.09758 50.74229),NaN,NaN,801621 POLYGON ((7.157885983913106 50.7227...,1162.0,Bonn Rosental,801621 ...
4807558,1006547199,2023-12-29 11:35:00,2023-12-29 12:05:00,e-scooter,0 days 00:30:00,8.172812,8172.812192,POINT (7.09716 50.74206),POINT (7.16496 50.71347),1162.0,Bonn Rosental,801621 POLYGON ((7.157885983913106 50.7227...,NaN,NaN,801621 ...
4807582,1006547223,2023-12-28 15:15:00,2023-12-28 15:20:00,e-scooter,0 days 00:05:00,0.115253,115.252520,POINT (7.14229 50.72758),POINT (7.14323 50.72803),1585.0,Bonn Küdinghoven,801621 POLYGON ((7.157885983913106 50.7227...,1585.0,Bonn Küdinghoven,801621 ...
4807597,1006547238,2023-12-28 16:00:00,2023-12-28 16:10:00,e-scooter,0 days 00:10:00,0.849810,849.809943,POINT (7.15428 50.74919),POINT (7.15354 50.74152),2304.0,Bonn Alte Schulstr.,801621 POLYGON ((7.157885983913106 50.7227...,2307.0,Bonn Holtorfer Str.,801621 ...


In [41]:
tier_trips.to_parquet("../Data/12_tier_part2.parquet", engine="pyarrow")